# Process synthetic enhancers sequencing data

In [ ]:
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/counting

## 00. Combine reads and 01. Fastqc

In [ ]:
module load FastQC/0.11.8-Java-1.8.0_162
# For group of samples
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
# Path to the different runs
    # Read 1 sequencing
NextSeq500_190426="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20190426/Demultiplexed/${line%Cheq*}*R1*" # Zeynep and Ibrahim
NovaSeq="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20190627/Demultiplexed/${line%Cheq*}*R1*" # Zeynep
NextSeq500_190925="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20190925/Demultiplexed/${line%Cheq*}*R1*" # Tiling A & B
NextSeq500_191010="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20191010/Demultiplexed/${line%Cheq*}*R1*" # Tiling A & B
NextSeq500_191126="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20191126/Demultiplexed/${line%Cheq*}*R1*" # Tiling A, B & AB
NovaSeq6000_200123="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200123/Demultiplexed/${line%Cheq*}*R1*" # Zeynep, Ibrahim, Tiling A & B
NovaSeq6000_210108="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20210108/Demultiplexed/${line%Cheq*}*R1*" # Tiling A & B

    # Read 2 sequencing
NextSeq500_190501="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20190501/Demultiplexed/${line%Cheq*}*R2*" # Zeynep and Ibrahim
NextSeq500_190724="/staging/leuven/stg_00002/lcb/ngs_runs/NextSeq500_20190724/Demultiplexed/${line%Cheq*}*R2*" # Zeynep, Tiling A & B

    #Concatenate R2 reads and run fastqc
# cat ${NextSeq500_190501} ${NextSeq500_190724} > 00.raw_reads/${line}_R2.fastq.gz
# fastqc -t 10 00.raw_reads/${line}_R2.fastq.gz -o 01.fastqc

    #Concatenate R1 reads and run fastqc
Novaseq6000_200611="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200611/Demultiplexed/${line%Cheq*}*R1*" # Liesbeth and Tiling A & B
Novaseq6000_200616="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200616/Demultiplexed/${line%Cheq*}*R1*" # Liesbeth and Tiling A & B
Novaseq6000_200730="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200730/Demultiplexed/${line%Cheq*}*R1*" # Tiling A & B
Novaseq6000_201013="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201013/Demultiplexed/${line%Cheq*}*R1*" # Liesbeth and Tiling A
NovaSeq_201126="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20201126/Demultiplexed/${line%CHEQ*}*R1*" # Tiling A & B
NovaSeq6000_210108="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20210108/Demultiplexed/${line%Cheq*}*R1*"

cat ${Novaseq6000_200611} ${Novaseq6000_200616} ${Novaseq6000_200730} ${Novaseq6000_201013} ${NovaSeq_201126} ${NovaSeq6000_210108} > 00.raw_reads/${line}_R1.fastq.gz
fastqc -t 10 00.raw_reads/${line}_R1.fastq.gz -o 01.fastqc
done

In [ ]:
# Total number of reads for R1 and R2 per sample
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "Combined number of reads for ${line}: " $(( $(zcat 00.raw_reads/${line}_R1.fastq.gz | wc -l) / 4 )) $(( $(zcat 00.raw_reads/${line}_R2.fastq.gz | wc -l) / 4 ))
done

## 02. Generate clean reads

In [ ]:
module use /data/leuven/software/biomed/skylake_centos7/2018a/modules/all/
module load SeqKit/0.10.2
module load cutadapt/1.18-foss-2018a-Python-3.7.0
start=`date +%s`
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
    # Do cutadapt trimming on R2 reads and then reverse complement
cutadapt -g GCGAATTAATTCGGGCCCCGGTCC...GATCGGCGCGCCTGC -e 0.15 -l 17 -m 17 -j 10 --discard-untrimmed \
         00.raw_reads/${line}_R2.fastq.gz | \
seqkit seq -r -p -o 02.cleaned_reads/${line}_R2.fastq.gz
    # Do cutadapt trimming on R1 reads
        # Adjust -e to 0.2 for the NovaSeq run because of low sequencing quality on 5 bases!
cutadapt -g ACTCGAGCCGAGCAGGCGCGCCGATC...GGACCGG -e 0.2 -l 17 -m 17 -j 10 --discard-untrimmed \
         -o 02.cleaned_reads/${line}_R1.fastq.gz \
         00.raw_reads/${line}_R1.fastq.gz
done
end=`date +%s`
echo $((end-start))

In [ ]:
module load fastp/0.20.0-foss-2018a
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
# Combine R1 and R2 then filter for Q30 reads
zcat 02.cleaned_reads/${line}_R*.fastq.gz | \
fastp --stdin -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 \
      -o 02.cleaned_reads/${line}_q30.fastq.gz
done

In [ ]:
# Total number of clean reads per sample
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "Number of reads after Q30 filtering for ${line}: " $(( $(zcat 02.cleaned_reads/${line}_q30.fastq.gz | wc -l) / 4 ))
done

## 03. Sequencing saturarion

In [ ]:
# Extract DNA sequences from fastq
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
zcat 02.cleaned_reads/${line}_q30.fastq.gz | sed -n '2~4p' > 03.sequencing_saturation/${line}_q30.txt
done

In [ ]:
# Subsample, count total and unique reads
start=`date +%s`

cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
file="03.sequencing_saturation/${line}_q30.txt"
nreads=$(cat $file | wc -l)
 # Start by removing parallel instructions file
rm 03.sequencing_saturation/parallel.txt
 # Remove any previoulsy created sat values for that file
rm ${file%_*}_sat.txt
    for i in $(seq 0 200000 $nreads)
    do
     # Print number of total and unique reads for every subsamples
    echo "echo $i \$(shuf -n $(($i)) $file | sort | uniq | wc -l) >> ${file%_*}_sat.txt" \
    >> 03.sequencing_saturation/parallel.txt
    done
cat 03.sequencing_saturation/parallel.txt | parallel -j 16
 # Put values in numerical order
cat ${file%_*}_sat.txt | sort -g > ${file%_*}_satsort.txt && mv ${file%_*}_satsort.txt ${file%_*}_sat.txt
done

end=`date +%s`
echo $((end-start))

## 10. Make BC count matrix

In [ ]:
# Generate BC count matrix for each samples
parallel \
"cat 03.sequencing_saturation/{}_q30.txt | sort | uniq -c | awk -F \" \" '{print \$2\"\t\"\$1}' \
   > 10.bc_count/{}_count.txt" ::: $(cat 00.raw_reads/CHEQseq_synth_names.txt)

# Count number of unique BC per sample
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo 10.bc_count/${line}_count.txt && cat 10.bc_count/${line}_count.txt | wc -l
done

In [ ]:
# Assign BC to enhancers
    # List of BC-Enhancer files 
OLSA="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_DavidTileA/20.coupling/CH1__b87bb6__CheqSeq_OLS_David_Pool_A_enhancer-barcode_S2_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz"
OLSB="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_DavidTileB/20.coupling/CH1__fb10ad__CheqSeq_OLS_David_Pool_B_enhancer-barcode_S5_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz"
OLSAB="/staging/leuven/stg_00002/lcb/lcb_projects/CSE/OLS_AB_BC-tile_assignment/20.coupling/CSE__8987a9__CheqSeq_enhancer-barcode_pool_OLS_AB_S1_readID_barcode_enhancerA_enhancerB_table.tsv.gz"
OLSZey="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_Zeynep/20.coupling/CH1__5b4298__CheqSeq_OLS_Zeynep_enhancer-barcode_S4_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz"
OLSIbr="/staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_Ibrahim/20.coupling/CH1__8395e4__CheqSeq_OLS_Ibrahim_enhancer-barcode_S3_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz"

    # OLS A samples
parallel \
"awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
   <( zcat ${OLSA} ) \
   <( cat 10.bc_count/{}_count.txt ) \
   | awk -F '\t' '{if (\$1) print \$0;}' \
   > 10.bc_count/{}_count_final.txt" ::: $(grep 'OLS_A' 00.raw_reads/CHEQseq_synth_names.txt)
   
   # OLS B samples
parallel \
"awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
   <( zcat ${OLSB} ) \
   <( cat 10.bc_count/{}_count.txt ) \
   | awk -F '\t' '{if (\$1) print \$0;}' \
   > 10.bc_count/{}_count_final.txt" ::: $(grep 'OLS_B' 00.raw_reads/CHEQseq_synth_names.txt)   
   
   # OLS AB samples
# parallel \
# "awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$2]=\$3\"__\"\$4 FS \$2;next}{ print a[\$1],\$2}' \
#    <( zcat ${OLSAB} ) \
#    <( cat 10.bc_count/{}_count.txt ) \
#    | awk -F '\t' '{if (\$1) print \$0;}' \
#    > 10.bc_count/{}_count_final.txt" ::: $(grep '_AB' 00.raw_reads/CHEQseq_synth_names.txt)     
  
   # OLS Zeynep samples
# parallel \
# "awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
#    <( zcat ${OLSZey} ) \
#    <( cat 10.bc_count/{}_count.txt ) \
#    | awk -F '\t' '{if (\$1) print \$0;}' \
#    > 10.bc_count/{}_count_final.txt" ::: $(grep '_Zeynep' 00.raw_reads/CHEQseq_synth_names.txt)   
   
   # OLS Ibrahim samples
# parallel \
# "awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$3]=\$2 FS \$3;next}{ print a[\$1],\$2}' \
#    <( zcat ${OLSIbr} ) \
#    <( cat 10.bc_count/{}_count.txt ) \
#    | awk -F '\t' '{if (\$1) print \$0;}' \
#    > 10.bc_count/{}_count_final.txt" ::: $(grep '_Ibrahim' 00.raw_reads/CHEQseq_synth_names.txt)  

In [ ]:
# Count number of BC associated to an enhancer
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
echo -e "${line}\t$(cat 10.bc_count/${line}_count_final.txt | wc -l)"
done

In [ ]:
# Coverage and Median number of BC per tile
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
file="10.bc_count/${line}_count_final.txt"
middle=$(($(awk '{print $1}' $file | sort | uniq -c | wc -l)/2))
echo $line
awk '{print $1}' $file | sort | uniq -c | wc -l
awk '{print $1}' $file | sort | uniq -c | awk '{print $1}' | sort -g | sed -n ${middle}p
done

In [ ]:
# Median number of reads per BC
cat 00.raw_reads/CHEQseq_synth_names.txt | while read line
do
file="10.bc_count/${line}_count_final.txt"
middle=$(($(cat $file | wc -l)/2))
echo ${line}
awk '{print $NF}' $file | sort -g | sed -n ${middle}p
done

In [ ]:
zcat /staging/leuven/stg_00002/lcb/dwmax/documents/aertslab/CH1/CHEQ-seq_OLS_DavidTileA/20.coupling/CH1__b87bb6__CheqSeq_OLS_David_Pool_A_enhancer-barcode_S2_enhancer_barcode_coupled_wo_empty_vectors_bc_rc.tsv.gz | head